# I51 : Define a Root Exception to Insulate Callers from APIs

- When you're defining a module's API, the exceptions you throw are just as mucjh a part of your interface as the functions and classes you define.

- Python has a built-in hierarchy of exceptions for the language and standard library. There's a draw to using the built-in exception types for reporting errors instead of defining your own new types. For example, you could raise a ValueError exception whenever an invalid parameter is passed to your function.

In [1]:
def determine_weight(volume, density):
    if density <= 0:
        raise ValueError("Density must be positive")
    pass

- In some cases, using ValueError makes sense, but for APIs it's much more powerful to define your own hierarchy of exceptions. You can do this by providing root Exception in your module. Then, have all other exceptions raised by that module inherit from the root exception.

In [2]:
class Error(Exception):
    """Base-class for all exceptions raised by this module."""
    
class InvalidDensityError(Error):
    """There was a problem with a provided density value."""

- Having a root exception in a module makes it easy for consumers of your API to catch all of the exceptions that you raise on purpose. For example, here a consumer of your API makes a function call with try/except statement that catches your root exception.

In [3]:
try:
    weight = my_module.determine_weight(1, -1)
except my_module.Error as e:
    logging.error('Unexpected error: $s', e)

NameError: name 'my_module' is not defined

- this try/except prevents your API's exceptions from propagating too far upward and breaking the calling program. It insulates the calling code from your API. This insulation has three helpful effects.

- Fist, root exceptions let callers understand when there's a problem with their usage of your API. If callers are using your API properly, they should catch the various exceptions that you deliberately raise. If they don't handle such an exception, it will propagate all the way up to the insulating except block that  catches your module's root exception. That block can bring the exception to the attention of the API consumer, giving them a chance to add proper handling of the exception type.

In [ ]:
try:
    weight = my_module.deterine_weight(1, -1)
except my_module.InvalidDensityError:
    weight = 0
except my_module.Error as e:
    logging.error("Bug in the calling code: %s", e)

- The second advantage of using root exceptions is that they can help find bugs in your APIs module's code. If your code only deliberately raises exceptions that you define within your module's hierarchy, then all other types of exceptions raised by your module must be the ones that you didn't intend to raise. These are bugs in your API's code.

- Using the try/except statement above will not insulate API consumers from bugs in your API module's code. To do that, the caller needs to add another except block that catches Python's base Exception class. This allows the API consumer to detect when there's bug in the API module's implementation that needs to be fixed.

In [ ]:
try:
    weight = my_module.determine_weight(1, -1)
except my_module.InvalidDensityError:
    weight = 0
except my_module.Error as e:
    logging.error('Bug in the calling code: %', e)
except Exception as e:
    logging.error('Bug in the API code: %s', e)
    raise

- The third impact of using root exceptions is future-proofing your API. Over time, you amy want to expand your API to provide more specific exceptions in certain situations. For example, you could add an Exception subclass that indicates the error condition of supplying negative densities.

In [ ]:
class NegativeDensityError(InvalidDensityError):
    """A provided density value was negative."""
    
def determine_weight(volume, density):
    if density < 0:
        raise NegativeDensityError

- The callign code will continue to work exactly as before because it already catches InvalidDensityError exceptions (the parent class of NegativeDensityError). In the future, the caller could decide to special-case the new type of exception and change its behavior accordingly.

In [ ]:
try:
    weight = my_module.determine_weight(1, -1)
except my_module.NegativeDensityError as e:
    raise ValueError('Must supply non-negative density') from e
except my_module.InvalidDensityError:
    weight = 0
except my_module.Error as e:
    logging.error('Bug in the calling code: %s', e)
except Exception as e:
    logging.error('Bug in the API code: %s', e)
    raise
    

- You can take API future-proofing further by providing a broader set of exceptions directly below the root exception. For exaple, imagine you had one set of errors related to calculating weights, another related to calculating volume, and a third related to calculating density.

In [ ]:
class WeightError(Error):
    """Base-class for weight calculation errors."""
    
class VolumeError(Error):
    """Base-class for volume calculation errors."""
    
class DensityError(Error):
    """Base-class for density calculation errors."""
    

- Specific exceptions would inherit from these general exceptions. Each intermediate exception acts as its own kind of root exception. This makes it easier to insulate layers of calling code from API code based ob broad functionality. This is much better than having all callers catch a long list of very specific Exception subclasses.

## Things to Remember
- Defining root exceptions for your modules allows API consumers to insulate themselves from your API.
- Catching root exceptions can help you find bugs in code that consumes an API.
- Catching the PYthon exception base class can help you find bugs in API implementations. 
- Intermediate root exceptions let you add more specific types of exceptions in the future without breaking your API consumers.

# I52: Know How to Break Circular Dependencies

- Inevitably, while you're collaborating with others, you'll find a mutual interdependency between modules, It can even happen while you work by yourself on the various parts of a single program.

- For example, say you want your GUI application to show a dialog box for choosing where to save a document. The data displayed by the dialog could be specified through arguments to your event handlers. But the dialog also needs to read global state, like user preferences, to know how to render properly.


In [ ]:
import app
class Dialog(object):
    def __init__(self, save_dir):
        self.save_dir = save_dir
    pass

save_dialot = Dialog(app.prefs.get('save_dir'))

def show():
    pass

- The problem is that the app module that contains the prefs object also imports the dialog class in order to show the dialog on program start.

In [ ]:
import dialog

class Prefs(object):
    pass 
def get(self, name):
    pass

prefs = Prefs()
dialog.show()

- It's a circular dependency. If you try to use the app module from your main program, you'll get an exception when you import it.

- To understand what's happening here, you need to know the details of Python's import machinery. When a module is imported, here's what Python actually does in depth-first order:

- Searches for your module in locations from sys.path
- Loads the code from the module and ensures that it compiles
- Creates a corresponding empty module object
- inserts the module into sys.modules
- Runs the code in the module object to define its contents

- The problem with a circular dependency is that the attributes of a module aren't defined until the code for those attributes has executed . But the module can be loaded with the import statement immediately after it's inserted into sys.modules.
- In the example, above, the app module imports dialog before defining anything. Then, the dialog module imports app. Since app still hasn't finished running the app module is just an empty shell. The AttributeError is raised because the code that defines prefs hasn't run yet.

- The best solution to this problem is to refactor your code so that the prefs data structure is at the bottom of the dependency tree. Then, both app and dialog can import the same uility module and avoid any circular dependencies. But such a clear division isn't always possible or could require too much refactoring to be worth the effort.
- There are three other ways to break circular dependencies."

** Reordering Imports **

- The first approach is to change the order of imports. For example, if you import the dialog module toward the bottom of the app module, after its contents have run the AttributeError goes away.

In [ ]:
class Prefs(object):
    pass
prefs = Prefs()

import dialog 
dialog.show()

- This works because, when the dialog module is loaded late, its recursive import of app will find the app.prefs has already been defined.
- Although this avoids the AttributeError, it goes against the PEP8 style guide. The style guide suggests that you always put imports at the top of your Python files. This makes your module's dependencies clear to new readers of the code. It also ensures that any module you depend on is in scope and available to all the code in your module.

- Having imports later in a file can be brittle and can cause small changes in the ordering of your code to break the module entirely. Thus, you should avoid reordering to solve your circular dependency issues.

** Import, Configure, Run **

- A second solution to the circular imports problem is to have your modules minimize side effects at import time. You have your modules only define functions, classes, and constants. You avoid actually running any functions at import time. Then, you have each module provide a configure function that you call once all other modules have finished importing. The purpose of configure is to prepare each module's state by accssing the attributes of other modules. You run configure after all modules have been imported, so all attributes must be defined.

- Here, I redefine the dialog module to only access the prefs object when configure is called:

In [ ]:
import app

class Dialog(object):
    pass

save_dialog = Dialog()

def show():
    pass

def configure():
    save_dialog.save_dir = app.prefs.get('save_dir')

- I also redefine the app module to not run any activities on import.

In [4]:
import dialog 

class Prefs(object):
    pass

prefs = Prefs()

def configure():
    pass

ModuleNotFoundError: No module named 'dialog'

- Finally, the main module has three disticnt phases of execution: import everything configure everything, and run the first activity.

In [ ]:
import app
import dialog

app.configure()
dialog.configure()


dialog.show()

- This works well in many situations and enables patterns like *dependency injection*. But sometimes it can be difficult to structure your code so that an explicit configure step is possible. Having two distinct phases within a module can make your code harder to read because it separates the definition of objects from their configuration.

** Dynamic import **

- The third solution to the circular imports problem is to use an import statement within a function or method. This it called a *dynamic import* because the module import happens while the program is running, not while the program is first starting up and initializing its modules.

- Here, I redefine the dialog module to use a dynamic import. The dialog.show function imports the app module at runtime instead of the dialog module importing app at initialization time.

In [ ]:
class Dialog(object):
    pass

save_dialog = Dialog()

def show():
    import app
    save_dialog.save_dir = app.prefs.get('save_dir')
    pass

In [ ]:
import dialog

class Prefs(object):
    pass

prefs = Prefs()
dialog.show()

- This approach has a similar effect to the import, configure, and run steps from before. The defference is that this requires no structural changes to the way the modules are defined and imported. You're simply delaying the circular import until the moment you must access the other module. At that point, you can be pretty sure that all other modules have already been initialized.

- In general, it's good to avoid dynamic imports like this. The cost of the import statement is not negligible and can be especially bad in tight loops. By delaying execution, dynamic imports also set you up for program has started running . However, these dowonsides are often better than the alternative of resturcturing your entire program.

## Things to Remember
- Circular dependencies happen when two modules must call into each other at import time. They can cause your program to crash at startup.
- The best way to break circular dependency is refactoring mutual dependencies into a separate module at the bottom of the dependency tree.
- Dynamic imports are the simplest solutioin for breaking a circular dependency between modules while minimizing refactoring and complexity.

# I53 : Use Virtual Environments for Isolated and Reproducible Dependencies

- Building larger and more complex programs often leads you to rely on various packages from the Python community. You'll find yourself running pip to install packages like pytz, numpy, and many others.

** Skip virtual env **

# 8. Production

- Putting a Python program to use requires moving it from a development environment to a production environment. Supporting sidparate configurations like this can be a challenge. Making programs that are dependable in multiple situations is just as important as making programs with correct functionality.

- The goal is to *productionize* your Python programs and make them bulletproof while they're in use. Python has built-in modules that aid in hardening your programs. Is provides facilities for debugging, optimizing, and testing to maximize the quality and performance of your programs at runtime.

# I54 : Consider Module-Scoped Code to Configure Deployment Environments.

- A development environment is a configuration in which your program runs. Every program has at least one deployment environment, the *production environment*. The goal of writing a program in the first place is to put is to work in the production environment and achieve some kind of outcome.

- Writing or modifying a program requires being able to run it on the computer you use for developing. The confituration of your *development environment* may be much different from your production environment. For example, you may be writing a program for supercomputers using a Linux workstation.

- Tools like pyvenv make it easy to ensure that all environments have the same Python packages insatalled. The trouble is that production environments often require many external assumptions that are hard to reproduce in development environments.

- For example, say you want to run your program in a web server container and give it access to a database. This means that every time you want to modify your program's code, you nned to run a server container, the database set up properly, and your program needs the password for access. That's a very high cost if all you're trying to do is verify that a one-line change your porgram works correctly.

- The best way to work around these issues is to override parts of your program at startup time to provide different functionality depending on the deployment environment. For example, you could have two different \_\_main\_\_ files, one for production and one for development.

In [ ]:
TESTING = True
import db_connection
db = db_connection.Database()

TESTING = False
import db_connection
db = db_connection.Database()

- The only difference between the two files is the value of the TESTING constant. Other modules is your program can then import the \_\_main\_\_module and use the value of TESTIGN to decide how they define their own attributes.

In [ ]:
import __main__

class TestingDatebase(object):
    pass

class RealDatabase(object):
    pass

if __main__.TESTING:
    Database = TestingDatabase
else:
    Database = RealDatabase

- The key behavior to notice here is that code running in module scope is just normal Python code. You can use an if statement at the module level to decide how the module will define names. This makes it easy to tailor modules to your various deployment environments. You can avoid having to reproduce costly assumptions like database configurations when they aren't needed. You can inject fake or mock implementations that ease interactive development and testing.

- This approach can be used for more than working around external assumptions. For example, if you know that your program must work differently based on its host platform, you can inspect the sys module before defining top-level constructs in a module

In [ ]:
import sys 

class Win32Database(object):
    pass

class PosixDatabase(object):
    pass

if sys.platform.startswith('win32'):
    Database = Win32Database
else:
    Databse = PosixDatabase

- Similarly, you can use environment variables from os.environ to guide your module definitions.

## Things to Remember

- Programs often need to run in multiple deployment environments that each have unique assumptions and configurations.
- You can tailor a module's contents to different deployment environments by using normal Python statements in module scope.
- Module contents can be the product of any external condition, including host introspection through the sys and os modules.